In [70]:
import google.generativeai as genai
import requests
from typing import List, Union
from fuzzywuzzy import process   
import pandas as pd

In [ ]:
import os
API_KEY = os.environ.get('GOOGLE_API_KEY')

In [5]:
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

In [6]:
history = []
chatdialog = model.start_chat(history=history)

In [7]:
user_message = "Hi, there. My name is Jiyar."

In [8]:
response = chatdialog.send_message(user_message)

In [16]:
# Use print even when in a notebook to handle newlines
print(response.text)

Hi Jiyar, it's nice to meet you! 😊  What can I do for you today? 



In [48]:
prompt = """Visa mig polisrapport för Stockholm perioden 15-20 juli"""
prompt_english = """Show me police reports for Stockholm for the period 15-20 July"""

In [80]:
prompt_swedish = """Visa mig polisens händelser för Stockholm."""
prompt_english = """Show me the reports from the Police for Stockholm."""

In [81]:
def police_feed(target_url: str = "https://polisen.se/api/events"):
    """
    This function calls an API for reported events by the Swedish Police Agency. Also called 'Polisen'.
    The call contains events of crimes split by type. When asked for the events you should look inside the pandas dataframe.

    Args:
        target_url: the url for the API

    Returns:
        events: a pandas dataframe if status_code = 200 else return response.status_code
    """
    response = requests.get(target_url)

    if response.status_code == 200:
        events = response.json()
        events = pd.json_normalize(events)
        events = pd.concat([events, events["location.gps"].str.split(",", expand=True).rename(columns={0:"lat", 1:"lon"})], axis=1)
        events["datetime"] = pd.to_datetime(pd.to_datetime(events.datetime, utc=False).dt.strftime('%Y-%m-%d'), format='%Y-%m-%d')

        return events
    
global police_events
police_events = police_feed()

def get_closest_match(query, choices):
    """
    Find the closest match for the query in the list of choices using fuzzy matching.
    
    Args:
        query (str): The query string to match.
        choices (list of str): The list of possible matches.
        
    Returns:
        str: The closest matching string from choices.
    """
    match, score = process.extractOne(query, choices)
    return match
    
def get_police_events(crime_type: Union[List[str]]=[], location_name: Union[List[str]]=[], crime_date: Union[List[str]]=[]):
    """
    Function that grabs crime reports, crime feed, police feed from Swedish Police, also known as Polisen.
    This allows the requester to get the type of crime from the Police events dataframe, filter on locations, get the latitude and longitude,
    the summary of the event, and get the date as well. The function is filtering regardless if provided with something or not.

    Args:
        crime_type: the type of crime they want to filter on. If they don't provide a filter then they want the whole dataframe.
        location_name: the location they want to filter on
        crime_date: the date they want to filter on. In the format YYYY-MM-DD as a string.
    
    Returns:
        filtered_data: a filtered dataframe
    """
    global police_events
    police_df = police_events.copy().drop(columns="id")
    
    if crime_type:
        police_df = police_df[police_df.type.isin(crime_type)]
    
    if location_name:
        unique_locations = police_df["location.name"].unique().tolist()
        matched_locations = [get_closest_match(loc, unique_locations) for loc in location_name]
        police_df = police_df[police_df["location.name"].isin(matched_locations)]
    
    if crime_date:
        police_df['datetime'] = pd.to_datetime(police_df['datetime']).dt.strftime('%Y-%m-%d')
        police_df = police_df[police_df["datetime"].isin(crime_date)]

    return police_df.to_json(force_ascii=False)

    

In [82]:
police_events

,id,datetime,name,summary,url,type,location.name,location.gps,lat,lon
0,556413,2024-11-06,"06 november 10.34, Trafikkontroll, Västernorrl...","Västernorrland, ett urval av polisens arbete i...",/aktuellt/handelser/2024/november/6/06-novembe...,Trafikkontroll,Västernorrlands län,"63.427647,17.729244",63.427647,17.729244
1,556412,2024-11-06,"06 november 10.29, Trafikolycka, Kalmar","Lastbil och personbilar i olycka, Kalmar.",/aktuellt/handelser/2024/november/6/06-novembe...,Trafikolycka,Kalmar,"56.663445,16.356779",56.663445,16.356779
2,556411,2024-11-06,"06 november 10.02, Trafikkontroll, Jämtlands län","Jämtland, ett urval av polisens arbete i trafi...",/aktuellt/handelser/2024/november/6/06-novembe...,Trafikkontroll,Jämtlands län,"63.171192,14.95918",63.171192,14.95918
3,556404,2024-11-06,"06 november 09.19, Arbetsplatsolycka, Ronneby","Olycka på kyrkogård, Möljeryd.",/aktuellt/handelser/2024/november/6/06-novembe...,Arbetsplatsolycka,Ronneby,"56.210434,15.276023",56.210434,15.276023
4,556407,2024-11-06,"06 november 10.00, Trafikkontroll, Norrbottens...","Norrbotten, resultat efter trafikkontroller un...",/aktuellt/handelser/2024/november/6/06-novembe...,Trafikkontroll,Norrbottens län,"66.830922,20.399197",66.830922,20.399197
...,...,...,...,...,...,...,...,...,...,...
495,555572,2024-10-30,"30 oktober 07.31, Sammanfattning natt, Dalarna...",Det har polisiärt sett varit en lugn natt i lä...,/aktuellt/handelser/2024/oktober/30/30-oktober...,Sammanfattning natt,Dalarnas län,"61.091701,14.666365",61.091701,14.666365
496,555571,2024-10-30,"30 oktober 07.31, Sammanfattning natt, Örebro län",Nedan följer en sammanfattning av natten.,/aktuellt/handelser/2024/oktober/30/30-oktober...,Sammanfattning natt,Örebro län,"59.535036,15.006573",59.535036,15.006573
497,555568,2024-10-30,"30 oktober 02.10, Skottlossning, Sigtuna",Flera skott har avlossats mot en dörr i ett lä...,/aktuellt/handelser/2024/oktober/30/30-oktober...,Skottlossning,Sigtuna,"59.619146,17.723419",59.619146,17.723419
498,555567,2024-10-30,"30 oktober 07.00, Sammanfattning kväll och nat...",Dagens presstalesperson är på plats.,/aktuellt/handelser/2024/oktober/30/30-oktober...,Sammanfattning kväll och natt,Hallands län,"56.896681,12.803399",56.896681,12.803399


In [83]:
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-001', tools=[get_police_events])
history = []
chatdialog = model.start_chat(history=history, enable_automatic_function_calling=True)

In [84]:
result = chatdialog.send_message(prompt_english)

In [85]:
print(result.text)

The police have reported the following incidents in Stockholm:

* **November 6, 00:48**: A fire was started when a flammable object was thrown into a residence on the ground floor of an apartment building in Östermalm.
* **November 6, 00:45**: A taxi driver in Tallkrogen called the police to report a customer who had taken drugs in his car and then refused to pay for the ride.
* **November 5, 20:03**: A man assaulted his girlfriend.
* **November 5, 17:10**: A young woman and two male companions threatened and assaulted a teenage girl in Älvsjö.
* **November 5, 00:50**: A police patrol in Kungsholmen stopped and searched a man, resulting in a report being filed for possession of narcotics.
* **November 5, 01:45**: A break-in alarm was triggered at a property in Bredäng. Upon inspection, it was found that a window had been broken.
* **November 4, 14:25**: A cyclist collided with a taxi driver in Vasastaden.
* **November 2, 18:19**: A boy was hit by a car in central Stockholm.
* **Novembe

In [86]:
prompt = """Show the"""
result = chatdialog.send_message(prompt)
print(result.text)

Please provide me with the rest of your request. I need to know what information you'd like to see about the police reports in Stockholm. For example, you could ask for:

* **The types of crimes:** "Show me the types of crimes reported in Stockholm."
* **The dates of the incidents:** "Show me the dates of the incidents in Stockholm."
* **Specific details about an incident:** "Show me the details about the incident in Stockholm involving a taxi driver and a customer taking drugs."

Once you give me more context, I can provide you with the information you need. 



In [88]:
prompt = """Lista alla för stockholm"""
prompt_english = """List all for stockholm"""
result = chatdialog.send_message(prompt_english)
print(result.text)

Here are the police reports for Stockholm:

**Date** | **Time** | **Incident Type** | **Summary**
------- | -------- | -------- | --------
November 6 | 00:48 | Brand (Fire) | A flammable object was thrown into a residence on the ground floor of an apartment building in Östermalm.
November 6 | 00:45 | Bedrägeri (Fraud) | A taxi driver in Tallkrogen reported a customer who took drugs in his car and then refused to pay.
November 5 | 20:03 | Misshandel (Assault) | A man assaulted his girlfriend.
November 5 | 17:10 | Misshandel (Assault) | A young woman and two male companions threatened and assaulted a teenage girl in Älvsjö.
November 5 | 00:50 | Narkotikabrott (Drug offense) | A police patrol in Kungsholmen stopped and searched a man, resulting in a report for possession of narcotics.
November 5 | 01:45 | Skadegörelse (Damage to property) | A break-in alarm was triggered at a property in Bredäng. A window was found broken.
November 4 | 14:25 | Trafikolycka, personskada (Traffic accident, 

In [92]:
prompt = """Visa för Malmö, Linköping, Norrköping och Uppsala"""
prompt_english = """Show for Malmö and Linköping"""
result = chatdialog.send_message(prompt_english)
print(result.text)

Here are the police reports for Malmö and Linköping:

**Malmö**

**Date** | **Time** | **Incident Type** | **Summary**
------- | -------- | -------- | --------
November 6 | 06:25 | Misshandel (Assault) | Alarm regarding assault in an apartment.
November 6 | 02:55 | Stöld (Theft) | Man suspected of stealing a bicycle.
November 5 | 22:02 | Brand (Fire) | Car fire.
November 5 | 14:09 | Stöld (Theft) | Theft from a store and threats to an employee.
November 5 | 00:34 | Mord/dråp, försök (Attempted murder) | Argument between several people on Föreningsgatan.
November 4 | 23:32 | Brand (Fire) | Fire in an abandoned building in the city center.
November 4 | 21:16 | Brand (Fire) | Car fire.
November 4 | 16:21 | Trafikolycka (Traffic accident) | Cyclist and passenger car involved.
November 4 | 00:05 | Rån (Robbery) | Man arrested for robbery in Holma.
November 3 | 17:29 | Rån (Robbery) | A woman was robbed on Möllevången.
November 3 | 05:39 | Brand (Fire) | Car fire in Hyllie.
November 2 | 06:0

In [93]:
prompt = """Ser du några skillnader mellan brotten för alla platser du har tagit fram?"""
prompt = """Do you see any differences between the crimes you've provided for between the locations?"""
result = chatdialog.send_message(prompt)
print(result.text)

You're right to point that out! There are some interesting observations we can make from the police reports for Malmö and Linköping:

**Malmö:**

* **Higher frequency of violent crimes:**  Malmö shows a higher number of assaults (Misshandel), robberies (Rån), and even attempted murders (Mord/dråp, försök) compared to Linköping.
* **Recurring car fires:** There are multiple reports of car fires (Brand) in Malmö. This might indicate a pattern of arson or other related issues.
* **Diverse types of theft:** We see reports of theft from stores (Stöld), bicycle theft (Stöld), and even theft/burglary (Stöld/inbrott) in Malmö.

**Linköping:**

* **Fewer violent crimes:**  The reports for Linköping mainly focus on traffic accidents (Trafikolycka), fires (Brand), and one incident of embezzlement (Förskingring).
* **Emphasis on traffic accidents:** Linköping has a higher proportion of reports related to traffic accidents, possibly reflecting differences in traffic density or driving habits compar